# 🎭 Roop Batch Processor - Google Colab

Sistema optimizado para procesamiento por lotes de face swap en Tesla T4

## 📋 Instrucciones:
1. **Ejecuta la celda de Setup** (solo una vez)
2. **Sube archivos** en las celdas correspondientes
3. **Ejecuta el procesamiento** por lotes
4. **Descarga resultados**

## 🚀 PASO 1: Setup Inicial (Ejecutar solo UNA vez)

In [ ]:
%%bash
# Clone repository
if [ ! -d "RoopConClaude" ]; then
  git clone https://github.com/CDavidDv/RoopConClaude.git
  echo "✅ Repositorio clonado"
else
  echo "⏭️  Repositorio ya existe"
fi

In [ ]:
# Change to project directory
%cd RoopConClaude

# Install dependencies in optimized order
print("📦 Instalando dependencias optimizadas...")
print("⏳ Esto tomará 2-3 minutos...\n")

!pip install -q numpy==1.23.5
!pip install -q tensorflow==2.12.0
!pip install -q onnxruntime-gpu==1.15.1
!pip install -q opencv-python==4.8.0.74
!pip install -q onnx==1.14.0
!pip install -q insightface==0.7.3
!pip install -q customtkinter==5.2.0
!pip install -q gfpgan==1.3.8
!pip install -q psutil==5.9.5
!pip install -q opennsfw2==0.10.2
!pip install -q protobuf==4.23.4
!pip install -q tqdm==4.65.0

print("\n✅ Dependencias instaladas correctamente")

In [ ]:
# Verify CUDA availability
import onnxruntime as ort

providers = ort.get_available_providers()
print("🔍 Verificación de aceleración por hardware:\n")

if 'CUDAExecutionProvider' in providers:
    print("✅ CUDA disponible - Procesamiento acelerado por GPU")
elif 'TensorrtExecutionProvider' in providers:
    print("✅ TensorRT disponible - Procesamiento acelerado por GPU")
else:
    print("⚠️  Solo CPU disponible - El procesamiento será más lento")

print(f"\n📋 Proveedores disponibles: {providers}")

# Show GPU info
!nvidia-smi --query-gpu=name,memory.total,memory.free --format=csv

## 📤 PASO 2: Subir Archivos

In [ ]:
# Upload source image (reference face)
from google.colab import files
import shutil
import os

print("📸 Sube tu imagen de referencia (la cara que quieres usar)")
print("   Formatos soportados: JPG, PNG, BMP\n")

# Create source folder if it doesn't exist
os.makedirs('source', exist_ok=True)

# Clear existing files
for f in os.listdir('source'):
    if f != '.gitkeep':
        os.remove(os.path.join('source', f))

uploaded = files.upload()

for filename, content in uploaded.items():
    dst_path = os.path.join('source', filename)
    with open(dst_path, 'wb') as f:
        f.write(content)
    print(f"✅ Imagen guardada: {dst_path}")
    print(f"   Tamaño: {len(content)/1024:.1f} KB")

In [ ]:
# Upload target videos
from google.colab import files
import os

print("🎬 Sube los videos que quieres procesar")
print("   Formatos soportados: MP4, AVI, MOV, MKV")
print("   Puedes subir múltiples videos a la vez\n")

# Create inputVideos folder if it doesn't exist
os.makedirs('inputVideos', exist_ok=True)

# Clear existing files
for f in os.listdir('inputVideos'):
    if f != '.gitkeep':
        os.remove(os.path.join('inputVideos', f))

uploaded = files.upload()

print(f"\n📊 Videos subidos:")
for filename, content in uploaded.items():
    dst_path = os.path.join('inputVideos', filename)
    with open(dst_path, 'wb') as f:
        f.write(content)
    size_mb = len(content) / 1024 / 1024
    print(f"✅ {filename} ({size_mb:.1f} MB)")

## 🔍 PASO 3: Verificar Videos (Opcional pero Recomendado)

In [ ]:
# Check video validity before processing
!python check_videos.py

## 🎯 PASO 4: Procesamiento por Lotes

In [ ]:
# Run batch processing
!python runbatch.py

## 💾 PASO 5: Descargar Resultados

In [ ]:
# Download processed videos
from google.colab import files
import os
import glob

output_files = glob.glob('outputVideos/*.mp4')

if output_files:
    print(f"📹 Videos procesados encontrados: {len(output_files)}\n")

    for video_file in sorted(output_files):
        size_mb = os.path.getsize(video_file) / 1024 / 1024
        print(f"⬇️  Descargando: {os.path.basename(video_file)} ({size_mb:.1f} MB)")
        files.download(video_file)

    print(f"\n✅ Todos los videos descargados")
else:
    print("❌ No se encontraron videos procesados en outputVideos/")
    print("   Verifica que el procesamiento haya completado correctamente")

## 🔄 PASO 6: Limpiar y Procesar Nuevos Videos (Opcional)

In [ ]:
# Clean up for new batch
import shutil
import os

print("🧹 Limpiando archivos para nueva tanda...\n")

# Clear input videos
for f in os.listdir('inputVideos'):
    if f != '.gitkeep':
        os.remove(os.path.join('inputVideos', f))
        print(f"🗑️  Eliminado: inputVideos/{f}")

# Clear output videos
for f in os.listdir('outputVideos'):
    if f != '.gitkeep':
        os.remove(os.path.join('outputVideos', f))
        print(f"🗑️  Eliminado: outputVideos/{f}")

# Clear temp folder if exists
if os.path.exists('temp'):
    shutil.rmtree('temp')
    print(f"🗑️  Eliminada carpeta temporal")

print("\n✅ Limpieza completada")
print("📝 Ahora puedes volver al PASO 2 para subir nuevos videos")

## 🔧 Diagnóstico (Solo si hay problemas)

In [ ]:
# Run diagnostics
!python debug_single.py

In [ ]:
# Check disk space
!df -h | grep -E "Filesystem|/dev/root"

In [ ]:
# Update repository (if needed)
!git pull origin main